In [1]:
!pip install soundfile pesq pystoi
!pip install torchcodec
!pip install torchinfo

  Preparing metadata (setup.py) ... done
  Created wheel for pesq: filename=pesq-0.0.4-cp312-cp312-linux_x86_64.whl size=284125 sha256=5fd292a8a73e81aa91cda25d38451f84692029dd7d956f85d10d5ccc58e1825f
  Stored in directory: /root/.cache/pip/wheels/9b/d4/a4/9cf3512534cd47ce4a036d1593ee4013f2bf7509e631a147a3
Successfully built pesq
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 41.6 MB/s eta 0:00:00


In [28]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import torch
import random
from pathlib import Path
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import torchaudio
import torchaudio.transforms as T
import soundfile as sf
import os
import time
import librosa
from pesq import pesq
from pystoi import stoi


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
base_drive_path = Path("/content/drive/MyDrive/Signal_MVA")

In [6]:
# Définition des dossiers spécifiques (comme dans votre exemple)
degraded_dir = base_drive_path / "data" / "packet_loss" / "train"
clean_dir = base_drive_path / "data" / "voice_origin" / "train"

# 2. Lists to store the matched paths
dataset_records = []

# 3. Iterate through the "packet loss" folder
print(f"Scanning {degraded_dir}...")

files_found = 0
matches_found = 0

for degraded_file in degraded_dir.glob("*.wav"):
    files_found += 1
    filename = degraded_file.name  # Get just the name like "recording1.wav"

    # Construct the expected path for the clean file
    expected_clean_path = clean_dir / filename

    # 4. Check if the corresponding clean file exists
    if expected_clean_path.exists():
        # Append to our list
        dataset_records.append({
            "filename": filename,
            "packet_loss_path": str(degraded_file),
            "voice_origin_path": str(expected_clean_path)
        })
        matches_found += 1
    else:
        print(f"Warning: No match found for {filename} in clean directory.")

# 5. Create a DataFrame and save to CSV
df_train = pd.DataFrame(dataset_records)

# Save to CSV (common format for ML dataloaders)
output_csv = "audio_dataset_mapping_train.csv"
df_train.to_csv(output_csv, index=False)
# --- Summary ---
print("-" * 30)
print(f"Processing complete.")
print(f"Total files scanned: {files_found}")
print(f"Matched pairs found: {matches_found}")
print(f"Mapping saved to: {output_csv}")
print("-" * 30)

# Display the first few rows to verify
df_train.head()

Scanning /content/drive/MyDrive/Signal_MVA/data/packet_loss/train...
------------------------------
Processing complete.
Total files scanned: 2118
Matched pairs found: 2118
Mapping saved to: audio_dataset_mapping_train.csv
------------------------------


,filename,packet_loss_path,voice_origin_path
0,Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap21...,/content/drive/MyDrive/Signal_MVA/data/packet_...,/content/drive/MyDrive/Signal_MVA/data/voice_o...
1,Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap10...,/content/drive/MyDrive/Signal_MVA/data/packet_...,/content/drive/MyDrive/Signal_MVA/data/voice_o...
2,Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap22...,/content/drive/MyDrive/Signal_MVA/data/packet_...,/content/drive/MyDrive/Signal_MVA/data/voice_o...
3,Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap19...,/content/drive/MyDrive/Signal_MVA/data/packet_...,/content/drive/MyDrive/Signal_MVA/data/voice_o...
4,Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap12...,/content/drive/MyDrive/Signal_MVA/data/packet_...,/content/drive/MyDrive/Signal_MVA/data/voice_o...


In [7]:
# Définition des dossiers spécifiques (comme dans votre exemple)
degraded_dir = base_drive_path / "data" / "packet_loss" / "test"
clean_dir = base_drive_path / "data" / "voice_origin" / "test"

# 2. Lists to store the matched paths
dataset_records = []

# 3. Iterate through the "packet loss" folder
print(f"Scanning {degraded_dir}...")

files_found = 0
matches_found = 0

for degraded_file in degraded_dir.glob("*.wav"):
    files_found += 1
    filename = degraded_file.name  # Get just the name like "recording1.wav"

    # Construct the expected path for the clean file
    expected_clean_path = clean_dir / filename

    # 4. Check if the corresponding clean file exists
    if expected_clean_path.exists():
        # Append to our list
        dataset_records.append({
            "filename": filename,
            "packet_loss_path": str(degraded_file),
            "voice_origin_path": str(expected_clean_path)
        })
        matches_found += 1
    else:
        print(f"Warning: No match found for {filename} in clean directory.")

# 5. Create a DataFrame and save to CSV
df_test = pd.DataFrame(dataset_records)

# Save to CSV (common format for ML dataloaders)
output_csv = "audio_dataset_mapping_test.csv"
df_test.to_csv(output_csv, index=False)
# --- Summary ---
print("-" * 30)
print(f"Processing complete.")
print(f"Total files scanned: {files_found}")
print(f"Matched pairs found: {matches_found}")
print(f"Mapping saved to: {output_csv}")
print("-" * 30)

# Display the first few rows to verify
df_test.head()

Scanning /content/drive/MyDrive/Signal_MVA/data/packet_loss/test...
------------------------------
Processing complete.
Total files scanned: 782
Matched pairs found: 782
Mapping saved to: audio_dataset_mapping_test.csv
------------------------------


,filename,packet_loss_path,voice_origin_path
0,Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap17...,/content/drive/MyDrive/Signal_MVA/data/packet_...,/content/drive/MyDrive/Signal_MVA/data/voice_o...
1,Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap15...,/content/drive/MyDrive/Signal_MVA/data/packet_...,/content/drive/MyDrive/Signal_MVA/data/voice_o...
2,Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap03...,/content/drive/MyDrive/Signal_MVA/data/packet_...,/content/drive/MyDrive/Signal_MVA/data/voice_o...
3,Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap17...,/content/drive/MyDrive/Signal_MVA/data/packet_...,/content/drive/MyDrive/Signal_MVA/data/voice_o...
4,Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap07...,/content/drive/MyDrive/Signal_MVA/data/packet_...,/content/drive/MyDrive/Signal_MVA/data/voice_o...


In [8]:
print(f"df_train shape: {df_train.shape}")
print(f"df_test shape: {df_test.shape}")

df_train shape: (2118, 3)
df_test shape: (782, 3)


In [9]:
# listen a .wav file
import IPython.display as ipd
ipd.Audio(df_train.iloc[0]['packet_loss_path'])

In [10]:
ipd.Audio(df_train.iloc[0]['voice_origin_path'])

# U-Net mix


In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class DoubleConv2D(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),

            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.block(x)


class UNet2D(nn.Module):
    def __init__(self, in_channels=1, out_channels=1, base_channels=32):
        super().__init__()

        # Encoder
        self.down1 = DoubleConv2D(in_channels, base_channels)
        self.pool1 = nn.MaxPool2d(2)

        self.down2 = DoubleConv2D(base_channels, base_channels * 2)
        self.pool2 = nn.MaxPool2d(2)

        self.down3 = DoubleConv2D(base_channels * 2, base_channels * 4)
        self.pool3 = nn.MaxPool2d(2)

        self.down4 = DoubleConv2D(base_channels * 4, base_channels * 8)
        self.pool4 = nn.MaxPool2d(2)

        # Bottleneck
        self.bottleneck = DoubleConv2D(base_channels * 8, base_channels * 16)

        # Decoder
        self.uptrans1 = nn.ConvTranspose2d(base_channels * 16, base_channels * 8,
                                           kernel_size=2, stride=2)
        self.upconv1 = DoubleConv2D(base_channels * 16, base_channels * 8)

        self.uptrans2 = nn.ConvTranspose2d(base_channels * 8, base_channels * 4,
                                           kernel_size=2, stride=2)
        self.upconv2 = DoubleConv2D(base_channels * 8, base_channels * 4)

        self.uptrans3 = nn.ConvTranspose2d(base_channels * 4, base_channels * 2,
                                           kernel_size=2, stride=2)
        self.upconv3 = DoubleConv2D(base_channels * 4, base_channels * 2)

        self.uptrans4 = nn.ConvTranspose2d(base_channels * 2, base_channels,
                                           kernel_size=2, stride=2)
        self.upconv4 = DoubleConv2D(base_channels * 2, base_channels)

        # Output
        self.out_conv = nn.Conv2d(base_channels, out_channels, kernel_size=1)

    def forward(self, x):
        # Encoder
        x1 = self.down1(x)   # (B, C, H, W)
        x2 = self.pool1(x1)

        x3 = self.down2(x2)
        x4 = self.pool2(x3)

        x5 = self.down3(x4)
        x6 = self.pool3(x5)

        x7 = self.down4(x6)
        x8 = self.pool4(x7)

        # Bottleneck
        x9 = self.bottleneck(x8)

        # Decoder
        x = self.uptrans1(x9)
        x = self._pad_or_crop_2d(x, x7)
        x = torch.cat([x7, x], dim=1)
        x = self.upconv1(x)

        x = self.uptrans2(x)
        x = self._pad_or_crop_2d(x, x5)
        x = torch.cat([x5, x], dim=1)
        x = self.upconv2(x)

        x = self.uptrans3(x)
        x = self._pad_or_crop_2d(x, x3)
        x = torch.cat([x3, x], dim=1)
        x = self.upconv3(x)

        x = self.uptrans4(x)
        x = self._pad_or_crop_2d(x, x1)
        x = torch.cat([x1, x], dim=1)
        x = self.upconv4(x)

        out = self.out_conv(x)  # (B, 1, H, W)
        # pas d'activation → régression directe sur magnitude [0,1]
        # tu peux faire out = out.clamp(min=0.0) si tu veux forcer non-négatif
        out = out.clamp(min=0.0)
        return out

    @staticmethod
    def _pad_or_crop_2d(x, ref):
        """
        Ajuste x pour qu'il ait la même taille spatiale que ref (H, W).
        """
        _, _, H_ref, W_ref = ref.shape
        _, _, H, W = x.shape

        # Pad
        pad_bottom = max(0, H_ref - H)
        pad_right = max(0, W_ref - W)
        if pad_bottom > 0 or pad_right > 0:
            x = F.pad(x, (0, pad_right, 0, pad_bottom))  # (left, right, top, bottom)
            _, _, H, W = x.shape

        # Crop
        if H > H_ref:
            x = x[:, :, :H_ref, :]
        if W > W_ref:
            x = x[:, :, :, :W_ref]

        return x


In [22]:
from torchinfo import summary

model = UNet2D(
    in_channels=2,
    out_channels=1,
    base_channels=32
)

summary(
    model,
    input_size=(1, 2, 256, 256),  # (batch_size, channels, H, W)
    col_names=["input_size", "output_size", "num_params", "kernel_size"],
    depth=4,
    device="cpu"
)


Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Kernel Shape
UNet2D                                   [1, 2, 256, 256]          [1, 1, 256, 256]          --                        --
├─DoubleConv2D: 1-1                      [1, 2, 256, 256]          [1, 32, 256, 256]         --                        --
│    └─Sequential: 2-1                   [1, 2, 256, 256]          [1, 32, 256, 256]         --                        --
│    │    └─Conv2d: 3-1                  [1, 2, 256, 256]          [1, 32, 256, 256]         608                       [3, 3]
│    │    └─BatchNorm2d: 3-2             [1, 32, 256, 256]         [1, 32, 256, 256]         64                        --
│    │    └─ReLU: 3-3                    [1, 32, 256, 256]         [1, 32, 256, 256]         --                        --
│    │    └─Conv2d: 3-4                  [1, 32, 256, 256]         [1, 32, 256, 256]         9,248                     [3, 3]
│    │

In [18]:
def count_parameters(model):
    # Compte total des éléments (numel = number of elements)
    total_params = sum(p.numel() for p in model.parameters())

    # Compte seulement ceux qui ont requires_grad=True
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    print(f"--- Statistiques du Modèle ---")
    print(f"Total des paramètres : {total_params:,}") # La virgule ajoute un séparateur de milliers
    print(f"Paramètres entraînables : {trainable_params:,}")
    print(f"Poids estimé (float32) : {total_params * 4 / (1024**2):.2f} MB")

    return total_params

# --- Utilisation ---
model = UNet2D(in_channels=1, out_channels=1)
count_parameters(model)

--- Statistiques du Modèle ---
Total des paramètres : 7,765,409
Paramètres entraînables : 7,765,409
Poids estimé (float32) : 29.62 MB


7765409

In [30]:
class SpectrogramAudioDataset(Dataset):
    """
    Version améliorée : génère dynamiquement un signal 'packet loss' à partir du clean.
    """

    def __init__(
        self,
        csv_file,
        sample_rate=8000,
        num_samples=None,
        n_fft=512,
        hop_length=128,
        normalize_mag=False,
        return_waveform=False,
        simulate_packet_loss=True,
        loss_prob=0.05,      # probabilité qu'un paquet soit perdu
        loss_width=200       # taille du trou en samples
    ):
        if isinstance(csv_file, str):
            self.dataframe = pd.read_csv(csv_file)
        else:
            self.dataframe = csv_file

        self.sample_rate = sample_rate
        self.num_samples = num_samples
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.normalize_mag = normalize_mag
        self.return_waveform = return_waveform

        self.simulate_packet_loss = simulate_packet_loss
        self.loss_prob = loss_prob
        self.loss_width = loss_width

        self.window = torch.hann_window(self.n_fft)

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        clean_path = self.dataframe.iloc[idx]["voice_origin_path"]

        clean_np, sr_cln = sf.read(clean_path)
        clean = torch.from_numpy(clean_np).float()

        # 1. Shape → [1, T]
        clean = clean.unsqueeze(0) if clean.ndim == 1 else clean.t()
        if clean.size(0) > 1:
            clean = clean.mean(dim=0, keepdim=True)

        # 2. Resample
        if sr_cln != self.sample_rate:
            clean = torchaudio.functional.resample(clean, sr_cln, self.sample_rate)

        # 3. Fix length (pad/crop)
        if self.num_samples is not None:
            clean = self._fix_length(clean)

        # 4. Generate degraded version (packet loss simulation)
        if self.simulate_packet_loss:
            degraded, mask_time = self._simulate_packet_loss(clean.clone())
        else:
            degraded = clean.clone()
            mask_time = (degraded.abs() > 1e-6).float()


        # 5. Convert to spectrogram magnitude
        mag_deg,_ = self._wav_to_mag(degraded)
        mag_cln,mag_phase = self._wav_to_mag(clean)

        if self.normalize_mag:
          mag_deg = torch.log1p(mag_deg)
          mag_cln = torch.log1p(mag_cln)

        mask_stft = self._mask_time_to_stft(mask_time)


        if self.return_waveform:
            return torch.cat([torch.log(mag_deg + 1e-7), mask_stft], dim=0), torch.log(mag_cln+1e-7), mag_phase, degraded, clean, mask_stft


        return torch.cat([torch.log(mag_deg + 1e-7), mask_stft], dim=0), torch.log(mag_cln+1e-7), mag_phase

    # -------------------------------------------
    # Helpers
    # -------------------------------------------
    def _mask_time_to_stft(self, mask_time):
      """
      Convertit un masque temporel (1D, 1 x T) en masque STFT (1 x F x T_frames)

      mask_time: torch.Tensor de forme (1, T), valeurs 0 ou 1
      Retourne:
          mask_stft: torch.Tensor de forme (1, F, T_frames), valeurs 0 ou 1
      """
      # S'assurer que mask_time est sur le bon device
      mask_time = mask_time.to(self.window.device)

      # STFT du masque (absolu, pour propager les zéros)
      stft_mask = torch.stft(
          mask_time.squeeze(0),               # (T,)
          n_fft=self.n_fft,
          hop_length=self.hop_length,
          window=self.window,
          return_complex=True
      )

      # Absolu et binaire : frame = 1 si au moins un sample valide dans la frame
      mask_stft = (stft_mask.abs() > 0).float().unsqueeze(0)  # (1, F, T_frames)

      return mask_stft

    def _fix_length(self, wav):
        _, T = wav.shape
        if T > self.num_samples:
            return wav[:, :self.num_samples]
        elif T < self.num_samples:
            pad = self.num_samples - T
            return F.pad(wav, (0, pad))
        return wav

    def _simulate_packet_loss(self, wav):
        _, T = wav.shape
        mask = torch.ones_like(wav)

        i = 0
        while i < T:
            if random.random() < self.loss_prob:
                end = min(i + self.loss_width, T)
                wav[:, i:end] = 0.0
                mask[:, i:end] = 0.0
            i += self.loss_width

        return wav, mask

    def _wav_to_mag(self, wav):

        wav_1d = wav.squeeze(0)
        stft = torch.stft(
            wav_1d,
            n_fft=self.n_fft,
            hop_length=self.hop_length,
            window = self.window.to(wav.device),
            return_complex=True,
        )
        mag = stft.abs().unsqueeze(0)
        phase = torch.angle(stft).unsqueeze(0)
        return mag,phase


In [ ]:
import torch
import torchaudio
import pandas as pd
import os
from tqdm import tqdm

def compute_log_mag_stats(csv_path, sample_rate=8000, n_fft=512, hop_length=128, eps=1e-7, device='cpu'):
    """
    Parcourt tous les fichiers listés dans le CSV, calcule log-magnitude STFT
    et renvoie mean et std globales pour normalisation.

    Args:
        csv_path (str): chemin vers le CSV contenant au moins une colonne 'voice_origin_path'
        sample_rate (int): taux d'échantillonnage pour le chargement
        n_fft (int): taille de la fenêtre STFT
        hop_length (int): hop length STFT
        eps (float): petite valeur pour le log
        device (str): 'cpu' ou 'cuda'

    Returns:
        dataset_mean (float), dataset_std (float)
    """
    df = pd.read_csv(csv_path)
    sum_vals = 0.0
    sum_sq_diff = 0.0
    num_pixels = 0

    # Première passe pour la moyenne
    print("Calcul de la moyenne...")
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        wav_path = row['voice_origin_path']
        if not os.path.exists(wav_path):
            continue

        waveform, sr = torchaudio.load(wav_path)
        if sr != sample_rate:
            waveform = torchaudio.functional.resample(waveform, sr, sample_rate)

        # mono si nécessaire
        if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0, keepdim=True)

        waveform = waveform.to(device)

        # STFT
        window = torch.hann_window(n_fft, device=device)
        stft = torch.stft(
            waveform.squeeze(0),
            n_fft=n_fft,
            hop_length=hop_length,
            window=window,
            return_complex=True
        )

        mag = stft.abs()
        log_mag = torch.log(mag + eps)

        sum_vals += log_mag.sum().item()
        num_pixels += log_mag.numel()

    dataset_mean = sum_vals / num_pixels
    print(f"Mean computed: {dataset_mean:.6f}")

    # Deuxième passe pour std
    print("Calcul de l'écart-type...")
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        wav_path = row['voice_origin_path']
        if not os.path.exists(wav_path):
            continue

        waveform, sr = torchaudio.load(wav_path)
        if sr != sample_rate:
            waveform = torchaudio.functional.resample(waveform, sr, sample_rate)
        if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0, keepdim=True)
        waveform = waveform.to(device)

        window = torch.hann_window(n_fft, device=device)
        stft = torch.stft(
            waveform.squeeze(0),
            n_fft=n_fft,
            hop_length=hop_length,
            window=window,
            return_complex=True
        )

        mag = stft.abs()
        log_mag = torch.log(mag + eps)
        sum_sq_diff += ((log_mag - dataset_mean) ** 2).sum().item()

    dataset_std = (sum_sq_diff / num_pixels) ** 0.5
    print(f"Std computed: {dataset_std:.6f}")

    return dataset_mean, dataset_std
csv_path = "/content/drive/MyDrive/Signal_MVA/audio_dataset_mapping_train.csv"
mean, std = compute_log_mag_stats(csv_path, sample_rate=8000, n_fft=512, hop_length=128)
print("Dataset mean:", mean)
print("Dataset std:", std)


In [ ]:
# =========================
# Hyperparameters
# =========================
CSV_PATH = "/content/drive/MyDrive/Signal_MVA/audio_dataset_mapping_train.csv"
CSV_PATH_TEST = "/content/drive/MyDrive/Signal_MVA/aaudio_dataset_mapping_test.csv"

SAMPLE_RATE = 8000
DURATION_SECONDS = 10
NUM_SAMPLES = SAMPLE_RATE * DURATION_SECONDS  # 80000

BATCH_SIZE = 8          # un peu plus petit avec MR-STFT, à adapter à ta VRAM
NUM_WORKERS = 4
NUM_EPOCHS = 50
LR = 1e-4               # plus raisonnable pour ce setup

# Early stopping
PATIENCE = 7
MIN_DELTA = 1e-4
BEST_MODEL_PATH = "unet2d_spectro_best_mrstft.pt"

# Perte combinée
ALPHA_MAG = 1  # poids L1 sur spectro
BETA_STFT = 0   # poids MR-STFT
GAMMA_TIME = 0  # poids L1 temporelle


# =========================
# Utils
# =========================
def format_time(seconds: float) -> str:
    seconds = int(seconds)
    h = seconds // 3600
    m = (seconds % 3600) // 60
    s = seconds % 60
    if h > 0:
        return f"{h:d}h {m:02d}m {s:02d}s"
    else:
        return f"{m:d}m {s:02d}s"


class MultiResolutionSTFTLoss(nn.Module):
    """
    Multi-Resolution STFT loss, classique en speech enhancement.
    Corrèle bien avec PESQ/STOI.
    """
    def __init__(self,
                 fft_sizes=(256, 512, 1024),
                 hop_sizes=(64, 128, 256),
                 win_lengths=(256, 512, 1024)):
        super().__init__()
        assert len(fft_sizes) == len(hop_sizes) == len(win_lengths)
        self.fft_sizes = fft_sizes
        self.hop_sizes = hop_sizes
        self.win_lengths = win_lengths

    def forward(self, x, y):
        """
        x, y : (B, 1, T) waveforms
        """
        x = x.squeeze(1)  # (B, T)
        y = y.squeeze(1)

        total_sc = 0.0
        total_mag = 0.0
        n_scales = len(self.fft_sizes)

        for fft, hop, win in zip(self.fft_sizes, self.hop_sizes, self.win_lengths):
            window = torch.hann_window(win, device=x.device)

            X = torch.stft(x, n_fft=fft, hop_length=hop, win_length=win,
                           window=window, return_complex=True)
            Y = torch.stft(y, n_fft=fft, hop_length=hop, win_length=win,
                           window=window, return_complex=True)

            mag_X = X.abs()
            mag_Y = Y.abs()

            # Spectral convergence
            sc = torch.norm(mag_Y - mag_X, p='fro', dim=(-2, -1)) / (
                torch.norm(mag_Y, p='fro', dim=(-2, -1)) + 1e-9
            )

            # Log-magnitude L1
            log_X = torch.log(mag_X + 1e-7)
            log_Y = torch.log(mag_Y + 1e-7)
            mag_l1 = torch.mean(torch.abs(log_Y - log_X), dim=(-2, -1))

            total_sc += sc.mean()
            total_mag += mag_l1.mean()

        return (total_sc + total_mag) / n_scales

def reconstruct_waveform_hybride(pred_mag, degraded_wav, mask_stft,
                                 n_fft=512, hop_length=128, griffin_iters=32):
    """
    pred_mag: (B, 1, F, T) predicted magnitude
    degraded_wav: (B, 1, T) degraded waveform (for STFT initialization)
    mask_stft: (B, 1, F, T) 1=valid, 0=hole (indique où on a un signal valide)

    """
    eps = 1e-7
    pred_mag = torch.exp(pred_log_mag) - eps
    pred_mag = torch.clamp(pred_mag, min=0.0)

    B, _, F, T = pred_mag.shape
    pred_wavs = []
    window = torch.hann_window(n_fft, device=pred_mag.device)

    for b in range(B):
        mag_b = pred_mag[b, 0]
        mask_b = mask_stft[b, 0]

        # STFT du degraded pour init phase
        stft_deg = torch.stft(
            degraded_wav[b, 0],
            n_fft=n_fft,
            hop_length=hop_length,
            window=window,
            return_complex=True,
        )
        phase_init = stft_deg.angle()  # phase initiale à partir du signal disponible

        # phase hybride = phase initiale partout, car pas de clean dispo
        phase_h = phase_init

        stft_h = mag_b * torch.exp(1j * phase_h)

        # Griffin-Lim only in holes
        for _ in range(griffin_iters):
            wav_tmp = torch.istft(
                stft_h,
                n_fft=n_fft,
                hop_length=hop_length,
                window=window,
                length=degraded_wav.shape[-1]
            )
            stft_new = torch.stft(
                wav_tmp,
                n_fft=n_fft,
                hop_length=hop_length,
                window=window,
                return_complex=True
            )
            # on met à jour la phase uniquement dans les trous
            phase_h = mask_b * phase_h + (1 - mask_b) * torch.angle(stft_new)
            stft_h = mag_b * torch.exp(1j * phase_h)

        wav_rec = torch.istft(
            stft_h,
            n_fft=n_fft,
            hop_length=hop_length,
            window=window,
            length=degraded_wav.shape[-1]
        )

        pred_wavs.append(wav_rec.unsqueeze(0))

    return torch.stack(pred_wavs, dim=0)  # (B, 1, T)

# =========================
# Main training
# =========================
def main():
    # ----------- DEVICE -----------
    if torch.backends.mps.is_available():
        device = torch.device("mps")
    elif torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")

    print(f"Using device: {device}")

    pin_memory = False if device.type == "mps" else True
    num_workers = NUM_WORKERS

    # ----------- DATASETS -----------
    train_dataset = SpectrogramAudioDataset(
        CSV_PATH,
        sample_rate=SAMPLE_RATE,
        num_samples=NUM_SAMPLES,
        n_fft=512,
        hop_length=128,
        normalize_mag=False,      # important pour garder l'échelle pour la reconstruction
        return_waveform=True,
    )

    val_dataset = SpectrogramAudioDataset(
        CSV_PATH_TEST,
        sample_rate=SAMPLE_RATE,
        num_samples=NUM_SAMPLES,
        n_fft=512,
        hop_length=128,
        normalize_mag=False,
        return_waveform=True,
    )

    print(f"Training on {len(train_dataset)} samples, validating on {len(val_dataset)} samples.")

    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=num_workers,
        persistent_workers=True if num_workers > 0 else False,
        pin_memory=pin_memory,
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=num_workers,
        persistent_workers=True if num_workers > 0 else False,
        pin_memory=pin_memory,
    )

    # ----------- MODEL & LOSSES -----------
    model = UNet2D(in_channels=2, out_channels=1, base_channels=32).to(device)

    criterion_mag = nn.L1Loss()           # L1 sur magnitude spectro
    mrstft = MultiResolutionSTFTLoss().to(device)  # MR-STFT sur waveform

    optimizer = torch.optim.Adam(model.parameters(), lr=LR)

    train_losses = []
    val_losses = []

    best_val_loss = float("inf")
    epochs_no_improve = 0

    start_time = time.time()

    # ----------- TRAINING LOOP -----------
    for epoch in range(1, NUM_EPOCHS + 1):
        # ====== TRAIN ======
        model.train()
        running_train_loss = 0.0

        train_pbar = tqdm(
            train_loader,
            desc=f"Epoch {epoch}/{NUM_EPOCHS} [Train]",
            leave=False,
        )

        for mag_input, mag_cln, mag_phase, degraded, clean, mask_stft in train_pbar:
            mag_input = mag_input.to(device)
            mag_cln = mag_cln.to(device)
            mag_phase = mag_phase.to(device)
            degraded = degraded.to(device)
            clean = clean.to(device)
            mask_stft = mask_stft.to(device)

            optimizer.zero_grad()
            pred_mag = model(mag_input)


            loss_mag = criterion_mag(pred_mag, mag_cln)

            if BETA_STFT ==0 and GAMMA_TIME == 0:
                loss_stft = 0
                loss_time = 0
            else:
                wav_pred = reconstruct_waveform_hybride(pred_mag, mag_phase, degraded, mask_stft)
                loss_stft = mrstft(wav_pred, clean)
                loss_time = F.l1_loss(wav_pred, clean)

            loss = ALPHA_MAG * loss_mag + BETA_STFT * loss_stft + GAMMA_TIME * loss_time
            loss.backward()
            optimizer.step()


            batch_loss = loss.item()
            running_train_loss += batch_loss * mag_input.size(0)

            train_pbar.set_postfix({
                "loss": f"{batch_loss:.4f}",
                "L_mag": f"{loss_mag.item():.4f}",
                "L_stft": f"{loss_stft.item():.4f}",
                "L_time": f"{loss_time.item():.4f}",
            })

        epoch_train_loss = running_train_loss / len(train_dataset)
        train_losses.append(epoch_train_loss)

        # ====== VALIDATION ======
        model.eval()
        running_val_loss = 0.0

        val_pbar = tqdm(
            val_loader,
            desc=f"Epoch {epoch}/{NUM_EPOCHS} [Val]",
            leave=False,
        )

        with torch.no_grad():
            for mag_input, mag_cln, mag_phase, degraded, clean, mask_stft in val_pbar:
                mag_input = mag_input.to(device)
                mag_cln = mag_cln.to(device)
                mag_phase = mag_phase.to(device)
                degraded = degraded.to(device)
                clean = clean.to(device)
                mask_stft = mask_stft.to(device)

                pred_mag = model(mag_input)
                wav_pred = reconstruct_waveform_hybride(pred_mag, mag_phase, degraded, mask_stft)

                loss_mag = criterion_mag(pred_mag, mag_cln)

                if BETA_STFT ==0 and GAMMA_TIME == 0:
                    loss_stft = 0
                    loss_time = 0
                else:
                    wav_pred = reconstruct_waveform_hybride(pred_mag, mag_phase, degraded, mask_stft)
                    loss_stft = mrstft(wav_pred, clean)
                    loss_time = F.l1_loss(wav_pred, clean)

                loss = ALPHA_MAG * loss_mag + BETA_STFT * loss_stft + GAMMA_TIME * loss_time

                batch_loss = loss.item()
                running_val_loss += batch_loss * mag_input.size(0)

                val_pbar.set_postfix({
                    "loss": f"{batch_loss:.4f}",
                    "L_mag": f"{loss_mag.item():.4f}",
                    "L_stft": f"{loss_stft.item():.4f}",
                    "L_time": f"{loss_time.item():.4f}",
                })

        epoch_val_loss = running_val_loss / len(val_dataset)
        val_losses.append(epoch_val_loss)

        # ====== TIME / ETA ======
        elapsed = time.time() - start_time
        avg_epoch_time = elapsed / epoch
        remaining_epochs = NUM_EPOCHS - epoch
        eta = avg_epoch_time * remaining_epochs

        print(
            f"Epoch {epoch:02d}/{NUM_EPOCHS:02d} | "
            f"Train Loss: {epoch_train_loss:.4f} | "
            f"Val Loss: {epoch_val_loss:.4f} | "
            f"Elapsed: {format_time(elapsed)} | "
            f"ETA: {format_time(eta)}"
        )

        # ====== EARLY STOPPING ======
        if epoch_val_loss + MIN_DELTA < best_val_loss:
            best_val_loss = epoch_val_loss
            epochs_no_improve = 0
            torch.save(model.state_dict(), BEST_MODEL_PATH)
            print(f"  ✅ New best val loss: {best_val_loss:.6f} | model saved to {BEST_MODEL_PATH}")
        else:
            epochs_no_improve += 1
            print(f"  No improvement for {epochs_no_improve} epoch(s).")

            if epochs_no_improve >= PATIENCE:
                print(f"Early stopping triggered after {epoch} epochs.")
                break

    print("Training finished.")
    print(f"Best validation loss: {best_val_loss:.6f}")
    print(f"Best model saved at: {BEST_MODEL_PATH}")


if __name__ == "__main__":
    main()


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Using device: cuda
Training on 2118 samples, validating on 782 samples.


Epoch 01/50 | Train Loss: 0.0437 | Val Loss: 0.0282 | Elapsed: 5m 55s | ETA: 4h 50m 33s
  ✅ New best val loss: 0.028222 | model saved to unet2d_spectro_best_mrstft.pt


Epoch 02/50 | Train Loss: 0.0291 | Val Loss: 0.0299 | Elapsed: 10m 44s | ETA: 4h 17m 59s
  No improvement for 1 epoch(s).


Epoch 03/50 | Train Loss: 0.0278 | Val Loss: 0.0241 | Elapsed: 16m 03s | ETA: 4h 11m 33s
  ✅ New best val loss: 0.024062 | model saved to unet2d_spectro_best_mrstft.pt


...

Epoch 14/50 | Train Loss: 0.0187 | Val Loss: 0.0182 | Elapsed: 1h 07m 55s | ETA: 2h 54m 40s
  ✅ New best val loss: 0.018222 | model saved to unet2d_spectro_best_mrstft.pt


Epoch 15/50 | Train Loss: 0.0188 | Val Loss: 0.0180 | Elapsed: 1h 12m 33s | ETA: 2h 49m 17s
  ✅ New best val loss: 0.018002 | model saved to unet2d_spectro_best_mrstft.pt


Epoch 16/50 | Train Loss: 0.0182 | Val Loss: 0.0158 | Elapsed: 1h 17m 11s | ETA: 2h 44m 02s
  ✅ New best val loss: 0.015806 | model saved to unet2d_spectro_best_mrstft.pt


Epoch 17/50 | Train Loss: 0.0178 | Val Loss: 0.0162 | Elapsed: 1h 21m 48s | ETA: 2h 38m 48s
  No improvement for 1 epoch(s).


Epoch 18/50 [Train]:  62%|██████▏   | 163/265 [02:29<01:27,  1.17it/s, loss=0.0147, L_mag=0.0147, L_stft=0.6638, L_time=0.0020]

In [ ]:
import pandas as pd
df = pd.read_csv("audio_dataset_mapping_train.csv")
print("✅ My CSV Columns are:", df.columns.tolist())
print("First row example:\n", df.iloc[0])

✅ My CSV Columns are: ['filename', 'packet_loss_path', 'voice_origin_path']
First row example:
 filename             Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap22...
packet_loss_path     /content/drive/MyDrive/train_packet_loss/Jules...
voice_origin_path    /content/drive/MyDrive/train_origin/Jules_Vern...
Name: 0, dtype: object


# Evaluation

In [ ]:



# ---------------- CONFIG ----------------
CHECKPOINT_PATH = "/content/unet2d_spectro_best_mrstft.pt"
CSV_PATH_TEST = "/content/audio_dataset_mapping_test.csv"

OUTPUT_DIR = base_drive_path / "data" / "voice_origin" / "outputs_collab"
MODEL_ARGS = {'in_channels': 2, 'out_channels': 1, "base_channels": 32}

# paramètres STFT (doivent matcher le training)
SAMPLE_RATE_MODEL = 8000
N_FFT = 512
HOP_LENGTH = 128

# ---------------- HELPER FUNCTIONS ----------------

def load_model(checkpoint_path, model_args=None):
    """Loads the UNet2D architecture and weights."""
    if model_args is None:
        model_args = {}
    print(f"Loading model from {checkpoint_path}...")

    model = UNet2D(**model_args)

    if torch.cuda.is_available():
        device = torch.device('cuda')
        checkpoint = torch.load(checkpoint_path)
    else:
        device = torch.device('cpu')
        checkpoint = torch.load(checkpoint_path, map_location=device)

    if isinstance(checkpoint, dict) and 'state_dict' in checkpoint:
        model.load_state_dict(checkpoint['state_dict'])
    elif isinstance(checkpoint, dict):
        model.load_state_dict(checkpoint)
    else:
        model = checkpoint  # si tu as sauvegardé directement le modèle

    model.to(device)
    model.eval()
    return model, device


def wav_to_mag_and_phase(wav, n_fft=N_FFT, hop_length=HOP_LENGTH, device="cpu"):
    """
    wav: torch.Tensor (1, T) sur 'device'
    return:
        mag: (1, 1, F, T_frames)
        phase: (F, T_frames)
    """
    wav_1d = wav.squeeze(0)  # (T,)
    window = torch.hann_window(n_fft, device=device)

    stft = torch.stft(
        wav_1d,
        n_fft=n_fft,
        hop_length=hop_length,
        window=window,
        return_complex=True,
    )  # (F, T_frames)

    mag = stft.abs().unsqueeze(0).unsqueeze(0)  # (1, 1, F, T_frames)
    phase = stft.angle()  # (F, T_frames)
    return mag, phase

def predict(audio_path, output_path, model, device):
    """
    Inference sur un fichier .wav dégradé :
    1) chargement waveform
    2) STFT -> magnitude + phase
    3) passage par le UNet2D (sur magnitude)
    4) ISTFT avec phase dégradée -> waveform reconstruite
    5) sauvegarde .wav
    """
    # on charge à la fréquence du modèle
    waveform_np, sample_rate = librosa.load(audio_path, sr=SAMPLE_RATE_MODEL, mono=True)
    wav = torch.from_numpy(waveform_np).float().unsqueeze(0).to(device)  # (1, T)
    T = wav.shape[-1]

    # 1. spectro magnitude + phase
    mag_deg, phase_deg = wav_to_mag_and_phase(
        wav, n_fft=N_FFT, hop_length=HOP_LENGTH, device=device
    )  # mag_deg: (1, 1, F, T_frames)

    # 2. passage dans le UNet2D
    with torch.no_grad():
        pred_mag = model(mag_deg,)  # (1, 1, F, T_frames_pred)

    # masque temporel
    mask_time = (wav.abs() > 1e-6).float()
    window = torch.hann_window(N_FFT, device=device)
    stft_mask = torch.stft(mask_time.squeeze(0), n_fft=N_FFT, hop_length=HOP_LENGTH, window=window, return_complex=True)
    mask_stft = (stft_mask.abs() > 0).float().unsqueeze(0).unsqueeze(1)  # (1, 1, F, T_frames)

    wav_rec = reconstruct_waveform_hybride(
        pred_mag,
        wav,
        mask_stft,
        n_fft=N_FFT,
        hop_length=HOP_LENGTH,
        griffin_iters=32
    ).squeeze(0)

    # 4. sauvegarde à la même fs que pour PESQ/STOI (tu peux garder 8k, ils resampleront après)
    wav_rec_np = wav_rec.cpu().numpy().squeeze()

    sf.write(output_path, wav_rec_np, SAMPLE_RATE_MODEL)

    return output_path


def evaluate_pair(reference_path, estimated_path):
    """Calculates PESQ and STOI for a pair of audio files."""
    TARGET_SR = 16000

    try:
        ref_audio, _ = librosa.load(reference_path, sr=TARGET_SR, mono=True)
        est_audio, _ = librosa.load(estimated_path, sr=TARGET_SR, mono=True)
    except Exception:
        return np.nan, np.nan

    min_len = min(len(ref_audio), len(est_audio))
    ref_audio = ref_audio[:min_len]
    est_audio = est_audio[:min_len]

    # STOI
    try:
        stoi_score = stoi(ref_audio, est_audio, TARGET_SR, extended=False)
    except Exception:
        stoi_score = np.nan

    # PESQ
    try:
        pesq_score = pesq(TARGET_SR, ref_audio, est_audio, 'wb')
    except Exception:
        pesq_score = np.nan

    return pesq_score, stoi_score


# ---------------- MAIN ----------------

def main():
    if not os.path.exists(CHECKPOINT_PATH):
        print(f"❌ Error: Model file '{CHECKPOINT_PATH}' not found.")
        return

    model, device = load_model(CHECKPOINT_PATH, MODEL_ARGS)
    print(f"✅ Model loaded on {device}")

    os.makedirs(OUTPUT_DIR, exist_ok=True)

    if not os.path.exists(degraded_dir):
        print(f"❌ Error: Input directory '{degraded_dir}' not found.")
        return
    if not os.path.exists(clean_dir):
        print(f"❌ Error: Reference directory '{clean_dir}' not found.")
        return

    files = [f for f in os.listdir(degraded_dir) if f.lower().endswith('.wav')]
    print(f"📂 Found {len(files)} files to process in {degraded_dir}")

    results = []

    for filename in files:
        degraded_path = os.path.join(degraded_dir, filename)
        reference_path = os.path.join(clean_dir, filename)
        output_path = os.path.join(OUTPUT_DIR, f"rec_{filename}")

        # A. Predict (spectro -> UNet2D -> waveform)
        predict(degraded_path, output_path, model, device)

        # B. Evaluate if ref exists
        if os.path.exists(reference_path):
            p_score, s_score = evaluate_pair(reference_path, output_path)
            results.append({
                "Filename": filename,
                "PESQ": p_score,
                "STOI": s_score
            })
            print(f"  --> {filename} | PESQ: {p_score:.2f} | STOI: {s_score:.2f}")
        else:
            print(f"  ⚠️ Processed {filename}, but NO reference found in {clean_dir} (Skipping metrics).")

    if results:
        df = pd.DataFrame(results)
        avg_pesq = df["PESQ"].mean()
        avg_stoi = df["STOI"].mean()

        print("\n" + "="*30)
        print("       FINAL RESULTS       ")
        print("="*30)
        print(f"Average PESQ: {avg_pesq:.4f}")
        print(f"Average STOI: {avg_stoi:.4f}")
        print("="*30)

        csv_out = "evaluation_results_colab.csv"
        df.to_csv(csv_out, index=False)
        print(f"📝 Detailed results saved to {csv_out}")
    else:
        print("Done. No evaluation metrics generated (check if reference files exist).")


if __name__ == "__main__":
    main()


Loading model from /content/unet2d_spectro_best_mrstft.pt...
✅ Model loaded on cuda
📂 Found 782 files to process in /content/drive/MyDrive/test_packet_loss
  --> Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap03_49_.wav | PESQ: 1.20 | STOI: 0.75
  --> Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap03_65_.wav | PESQ: 1.33 | STOI: 0.75
  --> Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap03_4_.wav | PESQ: 1.22 | STOI: 0.71
  --> Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap03_59_.wav | PESQ: 1.28 | STOI: 0.75
  --> Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap20_73_.wav | PESQ: 1.16 | STOI: 0.65
  --> Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap11_6_.wav | PESQ: 1.16 | STOI: 0.71
  --> Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap11_150_.wav | PESQ: 1.13 | STOI: 0.65
  --> Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap11_74_.wav | PESQ: 1.34 | STOI: 0.66
  --> Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap11_8_.wav | PESQ: 1.16 | STOI: 0.63
  --> Jules_Verne_-_L_ile_mysterieuse_Partie1_Chap17_144_.wav |

# With the previous waveUnet

In [ ]:
# print average scores
avg_pesq = results_df_train['PESQ'].mean()
avg_stoi = results_df_train['STOI'].mean()
print(f"Average PESQ: {avg_pesq:.4f}")
print(f"Average STOI: {avg_stoi:.4f}")

Average PESQ: 1.1737
Average STOI: 0.6564
